- Creating trading signals based on fundamental technical analysis 
- Creating predictive models using linear regression methods - OLS (Ordinary leas square method )
- Manage Portfolio risk and return -> Mean-Variance Optimisation.

### Development of Alpha Trading Strategy
- Alpha model is an algorithm that transforms input data (price/volume, news, fundamental, etc.) into a vector, which is pro- portional to the money we want to hold in each instrument.

- The goal is to make profits while minimizing risk, and to develop a mathematical predictive formula by using appropriate information.

### Steps to Create Model

Alpha Logic →Information in the Form of Data → Idea → Mathemati- cal Expression → Apply Operations → Final Robust Alpha → Translate into Positions in Financial Instrument → Check Historical PnL.
 

-Alpha model Should have three types of values:
- Positive → long position 
- Negative → Short position 
- 0 → No position

### Point out of scope for this project :
1. Incoming events, hence will go with sector neutral strategy ( sum of position = 0)
2. Trading Cost
3. Seasonility 
4. Considring Stationary Series ( Constant Mean, Constant Variance and Time - independent autocovariance )

###
the accuracy/quality of an alpha’s predictions by metrics such as information ratio (IR) and information coefficient (IC).

### Predefined Parameters :
- Book Size  = $20M

### PNL Calculation
Ann_return: = ann_pnl/(booksize/2)

### Inputs Parameter Creation for Alpha Model
> Short‐term horizon, the stock price tends to revert back due to the overreaction of the traders
- Alpha_Strategy_1 = –(close (today) – close (5_days_ago)) /close (5_days_ago)
>> Assumption of above logic :  Price will revert back to price 5 Days ago

### Fundamental Analyis Point for Alpha 
- Implementation of the simple moving average
- Exponential moving average
-  Relative Strength of Indicator
- Standard Deviation 
- Momentum

In [15]:
import pandas as pd
import random
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplot, scatter, plot, axis
from scipy.stats import linregress
import ast 
import re
import seaborn as sns



import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.stats as stats

pd.set_option("display.notebook_repr_html", False)
pd.set_option("display.max_columns", 30)
pd.set_option("display.width", 80)
pd.set_option("precision", 2)

%matplotlib inline
pd.set_option("max_rows", 10)
pd.set_option("min_rows", 8)


In [69]:
start  = datetime.date(2010,1,1)
end  = datetime.date(2015,12,1)

In [70]:
x = ("AAPL","GOOGL")
df =web.DataReader(x ,"yahoo",start,end )

In [71]:
df_data = df.stack().swaplevel().sort_index()


In [72]:
df_data_Adj_Close = df_data.reset_index()[["Date","Symbols","Adj Close"]].pivot("Date","Symbols","Adj Close")

In [73]:
df_data_Adj_Close.rolling(7).mean().dropna(axis = 0)

Symbols      AAPL   GOOGL
Date                     
2010-01-11   6.54  305.75
2010-01-12   6.52  303.64
2010-01-13   6.51  300.80
2010-01-14   6.49  298.36
...           ...     ...
2015-11-25  27.30  765.50
2015-11-27  27.44  769.22
2015-11-30  27.47  769.62
2015-12-01  27.42  773.03

[1484 rows x 2 columns]

In [74]:
#df_data_Adj_Close[ "AAPL_Alpha"] = df_data_Adj_Close.AAPL.apply( lambda  x : -x)
#df_data_Adj_Close[ "GOOGL_Alpha"] = df_data_Adj_Close.AAPL.apply( lambda  x : -x)
#df_data_Adj_Close[["AAPL","AAPL_Alpha"]].plot()
#df_data_Adj_Close[ "AAPL_Alpha_Position" ] = df_data_Adj_Close.apply( lambda x :  x["AAPL_Alpha"] / (x["AAPL_Alpha"] + x["GOOGL_Alpha"]) , axis =1)
#df_data_Adj_Close[ "GOOGL_Alpha_Position" ] = df_data_Adj_Close.apply( lambda x :  (x["AAPL_Alpha_Position"] -1) , axis =1)

### Implementation Alpha Strategy : 1
> Short‐term horizon, the stock price tends to revert back due to the overreaction of the traders
- Alpha_Strategy_1 = –(close (today) – close (5_days_ago)) /close (5_days_ago)
>> Assumption of above logic :  Price will revert back to price 5 Days ago

In [75]:
start  = datetime.date(2010,1,1)
end  = datetime.date(2015,12,1)
df_data_Adj_Close = df_data.reset_index()[["Date","Symbols","Adj Close"]].pivot("Date","Symbols","Adj Close")

def alpha_strategy_1(today_close_price,previous_Close_price):
    
    return (-(today_close_price - previous_Close_price) /previous_Close_price)

In [76]:
df_data_Adj_Close["Alpha_Strategy_1_Shift_5_AAPL"] = df_data_Adj_Close["AAPL"].shift(5)
df_data_Adj_Close["Alpha_Strategy_1_Shift_5_GOOGL"] = df_data_Adj_Close["GOOGL"].shift(5)
df_data_Adj_Close["alpha_strategy_1_AAPL_Output"] = df_data_Adj_Close.dropna(axis = 0).apply( lambda x : alpha_strategy_1( x["AAPL"],x["Alpha_Strategy_1_Shift_5_AAPL"]),axis =1)
df_data_Adj_Close["alpha_strategy_1_GOOGL_Output"] = df_data_Adj_Close.dropna(axis = 0).apply( lambda x : alpha_strategy_1( x["GOOGL"],x["Alpha_Strategy_1_Shift_5_GOOGL"]),axis =1)

In [77]:
df_data_Adj_Close[["AAPL","GOOGL","alpha_strategy_1_AAPL_Output","alpha_strategy_1_GOOGL_Output"]].dropna(axis =0)

Symbols      AAPL   GOOGL  alpha_strategy_1_AAPL_Output  \
Date                                                      
2010-01-08   6.54  301.31                     -5.93e-03   
2010-01-11   6.48  300.86                      1.82e-02   
2010-01-12   6.41  295.54                      3.11e-02   
2010-01-13   6.50  293.84                      1.52e-03   
...           ...     ...                           ...   
2015-11-25  27.39  769.26                     -6.31e-03   
2015-11-27  27.33  771.97                      8.17e-03   
2015-11-30  27.45  762.85                      8.38e-03   
2015-12-01  27.22  783.79                      3.48e-03   

Symbols     alpha_strategy_1_GOOGL_Output  
Date                                       
2010-01-08                       2.90e-02  
2010-01-11                       4.09e-02  
2010-01-12                       5.37e-02  
2010-01-13                       3.48e-02  
...                                   ...  
2015-11-25                      -1.22e-02 

In [78]:
df_data_Adj_Close[["AAPL","Alpha_Strategy_1_Shift_5_AAPL","alpha_strategy_1_AAPL_Output"]].dropna(axis =0).astype('float').round(3)

Symbols      AAPL  Alpha_Strategy_1_Shift_5_AAPL  alpha_strategy_1_AAPL_Output
Date                                                                          
2010-01-08   6.54                           6.50                     -6.00e-03
2010-01-11   6.48                           6.61                      1.80e-02
2010-01-12   6.41                           6.62                      3.10e-02
2010-01-13   6.50                           6.51                      2.00e-03
...           ...                            ...                           ...
2015-11-25  27.39                          27.21                     -6.00e-03
2015-11-27  27.33                          27.56                      8.00e-03
2015-11-30  27.45                          27.68                      8.00e-03
2015-12-01  27.23                          27.32                      3.00e-03

[1485 rows x 3 columns]

In [79]:
## Annual Reutrn ??
## Information Ratio
## PNL
## Max Draw Down

In [80]:
## Fundamental Analysis   - Efficient Market Hypothesis 

In [81]:
df_data_1 = df.stack().swaplevel().sort_index()

In [84]:
df_data_1 = df_data_1.reset_index()

In [85]:
df_data_1import statistics as stats

Attributes Symbols       Date  Adj Close   Close    High     Low    Open  \
0             AAPL 2009-12-31       6.50    7.53    7.62    7.52    7.61   
1             AAPL 2010-01-04       6.60    7.64    7.66    7.59    7.62   
2             AAPL 2010-01-05       6.62    7.66    7.70    7.62    7.66   
3             AAPL 2010-01-06       6.51    7.53    7.69    7.53    7.66   
...            ...        ...        ...     ...     ...     ...     ...   
2976         GOOGL 2015-11-25     769.26  769.26  773.00  766.53  771.69   
2977         GOOGL 2015-11-27     771.97  771.97  774.00  767.00  769.16   
2978         GOOGL 2015-11-30     762.85  762.85  775.03  761.12  771.87   
2979         GOOGL 2015-12-01     783.79  783.79  785.00  765.95  766.94   

Attributes    Volume  
0           3.52e+08  
1           4.94e+08  
2           6.02e+08  
3           5.52e+08  
...              ...  
2976        1.18e+06  
2977        7.22e+05  
2978        1.68e+06  
2979        2.02e+06  

[2980 ro

In [86]:
import pandas as pd
import numpy as np
from pandas_datareader import data
start_date = '2014-01-01' 
end_date = '2018-01-01'

goog_data = data.DataReader('GOOG', 'yahoo', start_date, end_date) 


In [87]:
goog_data

               High      Low     Open    Close    Volume  Adj Close
Date                                                               
2013-12-31   558.41   551.06   554.04   558.26  2.73e+06     558.26
2014-01-02   556.79   552.06   555.65   554.48  3.66e+06     554.48
2014-01-03   556.38   550.40   555.42   550.44  3.35e+06     550.44
2014-01-06   557.34   551.15   554.43   556.57  3.55e+06     556.57
...             ...      ...      ...      ...       ...        ...
2017-12-26  1060.12  1050.20  1058.07  1056.74  7.61e+05    1056.74
2017-12-27  1058.37  1048.05  1057.39  1049.37  1.27e+06    1049.37
2017-12-28  1054.75  1044.77  1051.60  1048.14  8.37e+05    1048.14
2017-12-29  1049.70  1044.90  1046.72  1046.40  8.88e+05    1046.40

[1008 rows x 6 columns]

In [88]:
goog_data_signal = pd.DataFrame(index=goog_data.index) 
goog_data_signal['price'] = goog_data['Adj Close']

In [95]:
def trading_support_resistance(data, bin_width=20): 
    data['sup_tolerance'] = pd.Series(np.zeros(len(data))) 
    data['res_tolerance'] = pd.Series(np.zeros(len(data))) 
    data['sup_count'] = pd.Series(np.zeros(len(data))) 
    data['res_count'] = pd.Series(np.zeros(len(data)))
    data['sup'] = pd.Series(np.zeros(len(data))) 
    data['res'] = pd.Series(np.zeros(len(data))) 
    data['positions'] = pd.Series(np.zeros(len(data)))
    data['signal'] = pd.Series(np.zeros(len(data))) 
    in_support=0
    in_resistance=0
    
    for x in range((bin_width - 1) + bin_width, len(data)): 
        data_section = data[x - bin_width:x + 1] 
        support_level=min(data_section['price']) 
        resistance_level=max(data_section['price']) 
        range_level=resistance_level-support_level 
        data['res'][x]=resistance_level
        data['sup'][x]=support_level 
        data['sup_tolerance'][x]=support_level + 0.2 * range_level 
        data['res_tolerance'][x]=resistance_level - 0.2 * range_level
        if data['price'][x]>=data['res_tolerance'][x] and\ data['price'][x] <= data['res'][x]:
            in_resistance+=1
            data['res_count'][x]=in_resistance
            elif data['price'][x] <= data['sup_tolerance'][x] and \
            data['price'][x] >= data['sup'][x]:
            in_support += 1
            data['sup_count'][x] = in_support else:
            in_support=0
            in_resistance=0 if in_resistance>2:
            data['signal'][x]=1 elif in_support>2:
            data['signal'][x]=0 else:
            data['signal'][x] = data['signal'][x-1]

SyntaxError: unexpected character after line continuation character (<ipython-input-95-fb2a33a63bee>, line 22)

NameError: name 'bin_width' is not defined

In [97]:
goog_data_signal["SMA"] = goog_data_signal["price"].rolling(20).mean()

In [99]:
goog_data_signal.dropna(axis = 0, inplace = True)

In [101]:
goog_data_signal["EMA"] = goog_data_signal["price"].ewm(20).mean()

In [102]:
goog_data_signal

              price      SMA      EMA
Date                                 
2014-01-29   551.39   564.59   551.39
2014-01-30   565.58   564.95   558.66
2014-01-31   588.28   566.64   569.02
2014-02-03   564.60   567.35   567.83
...             ...      ...      ...
2017-12-26  1056.74  1040.98  1030.40
2017-12-27  1049.37  1041.08  1031.30
2017-12-28  1048.14  1042.40  1032.11
2017-12-29  1046.40  1043.65  1032.79

[989 rows x 3 columns]